# NASNetLarge

## Data Pipeline for CPU Mac

### Unreduced Image Size

In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import NASNetLarge
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import matplotlib.pyplot as plt
from tensorflow.keras.regularizers import l2


csv_folder_path = '/Users/shubhibhadauriya/Desktop/csv'  
image_base_path = '/Users/shubhibhadauriya/Desktop/images'


csv_files = [f for f in os.listdir(csv_folder_path) if f.endswith('.csv')]


all_filenames = []
all_labels = []


def find_image_file(csv_filename, image_dir):
    parts = csv_filename.split('/')
    timestamp_id = parts[-1].split('-')[0]
    corrected_timestamp_id = timestamp_id.replace(':', '/')
    for root, dirs, files in os.walk(image_dir):
        for file in files:
            if corrected_timestamp_id in file:
                return os.path.join(root, file)
    return None


def load_images(filenames, base_path):
    images = []
    for filename in filenames:
        img_path = find_image_file(filename, base_path)
        if img_path and os.path.exists(img_path):
            images.append(img_path)
        else:
            print(f"File not found for CSV entry: {filename}")
            images.append(None)  
    return np.array(images)


for csv_file in csv_files:
    
    csv_file_path = os.path.join(csv_folder_path, csv_file)
    data = pd.read_csv(csv_file_path)
    data = data.dropna()
    
    
    image_width, image_height = 640, 480
    data['lx'] = data['lx'] / image_width
    data['ly'] = data['ly'] / image_height
    data['rx'] = data['rx'] / image_width
    data['ry'] = data['ry'] / image_height

   
    features = ['lx', 'ly', 'rx', 'ry']

    
    filenames = data['filename'].values
    images = load_images(filenames, image_base_path)
    labels = data[features].values

    
    valid_indices = [i for i, img in enumerate(images) if img is not None]
    images = images[valid_indices]
    labels = labels[valid_indices]

    
    all_filenames.extend(images)
    all_labels.extend(labels)


all_filenames = tf.convert_to_tensor(all_filenames, dtype=tf.string)
all_labels = tf.convert_to_tensor(all_labels, dtype=tf.float32)


dataset = tf.data.Dataset.from_tensor_slices((all_filenames, all_labels))


dataset = dataset.shuffle(buffer_size=len(all_filenames), seed=42)


train_size = int(0.7 * len(all_filenames))
val_size = int(0.15 * len(all_filenames))
test_size = len(all_filenames) - train_size - val_size


train_dataset = dataset.take(train_size)
val_test_dataset = dataset.skip(train_size)
val_dataset = val_test_dataset.take(val_size)
test_dataset = val_test_dataset.skip(val_size)


def load_and_preprocess_image(filename, label):
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [480, 640])
    image = image / 255.0  
    return image, label


train_dataset = train_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)


train_dataset = train_dataset.cache().batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)  
val_dataset = val_dataset.cache().batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.cache().batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)


base_model = NASNetLarge(weights='imagenet', include_top=False, input_shape=(480, 640, 3))


for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x) 
x = Dropout(0.5)(x)  
x = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(x)
predictions = Dense(4, activation='linear')(x)


model = Model(inputs=base_model.input, outputs=predictions)


optimizer = Adam(learning_rate=1e-4)  
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])


early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)  
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-7)  
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')


history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=10,  
                    callbacks=[early_stopping, reduce_lr, checkpoint])


model.load_weights('best_model.keras')

# Evaluate on the test data
#test_loss, test_mae = model.evaluate(test_dataset)
#print(f"Test Loss: {test_loss}")
#print(f"Test MAE: {test_mae}")

# Plot history
def plot_history(history, model_name):
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['loss'], label='train_loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.title(f'{model_name} Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    plt.figure(figsize=(12, 6))
    plt.plot(history.history['mae'], label='train_mae')
    plt.plot(history.history['val_mae'], label='val_mae')
    plt.title(f'{model_name} MAE')
    plt.xlabel('Epochs')
    plt.ylabel('MAE')
    plt.legend()
    plt.show()

plot_history(history, 'NASNetLarge Fine-tuned Model')


2024-08-27 15:29:50.479075: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


343610240/343610240 ━━━━━━━━━━━━━━━━━━━━ 34s 0us/step
Epoch 1/10
 1/71 ━━━━━━━━━━━━━━━━━━━━ 5:45:17 296s/step - loss: 4.1058 - mae: 0.7968

KeyboardInterrupt: 

### Reduced Size

In [2]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import NASNetLarge
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import matplotlib.pyplot as plt
from tensorflow.keras.regularizers import l2


csv_folder_path = '/Users/shubhibhadauriya/Desktop/csv'  
image_base_path = '/Users/shubhibhadauriya/Desktop/images'


csv_files = [f for f in os.listdir(csv_folder_path) if f.endswith('.csv')]


all_filenames = []
all_labels = []


def find_image_file(csv_filename, image_dir):
    parts = csv_filename.split('/')
    timestamp_id = parts[-1].split('-')[0]
    corrected_timestamp_id = timestamp_id.replace(':', '/')
    for root, dirs, files in os.walk(image_dir):
        for file in files:
            if corrected_timestamp_id in file:
                return os.path.join(root, file)
    return None


def load_images(filenames, base_path):
    images = []
    for filename in filenames:
        img_path = find_image_file(filename, base_path)
        if img_path and os.path.exists(img_path):
            images.append(img_path)
        else:
            print(f"File not found for CSV entry: {filename}")
            images.append(None)  
    return np.array(images)


for csv_file in csv_files:
    
    csv_file_path = os.path.join(csv_folder_path, csv_file)
    data = pd.read_csv(csv_file_path)
    data = data.dropna()

    # Normalize coordinates
    image_width, image_height = 320, 240  # Reduce image size
    data['lx'] = data['lx'] / image_width
    data['ly'] = data['ly'] / image_height
    data['rx'] = data['rx'] / image_width
    data['ry'] = data['ry'] / image_height

   
    features = ['lx', 'ly', 'rx', 'ry']

    
    filenames = data['filename'].values
    images = load_images(filenames, image_base_path)
    labels = data[features].values

    
    valid_indices = [i for i, img in enumerate(images) if img is not None]
    images = images[valid_indices]
    labels = labels[valid_indices]

    
    all_filenames.extend(images)
    all_labels.extend(labels)

# Convert to TensorFlow tensors
all_filenames = tf.convert_to_tensor(all_filenames, dtype=tf.string)
all_labels = tf.convert_to_tensor(all_labels, dtype=tf.float32)


dataset = tf.data.Dataset.from_tensor_slices((all_filenames, all_labels))


dataset = dataset.shuffle(buffer_size=len(all_filenames), seed=38)


train_size = int(0.7 * len(all_filenames))
val_size = int(0.15 * len(all_filenames))
test_size = len(all_filenames) - train_size - val_size


train_dataset = dataset.take(train_size)
val_test_dataset = dataset.skip(train_size)
val_dataset = val_test_dataset.take(val_size)
test_dataset = val_test_dataset.skip(val_size)


def load_and_preprocess_image(filename, label):
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [240, 320])  
    image = image / 255.0  
    return image, label


train_dataset = train_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)


train_dataset = train_dataset.cache().batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)  
val_dataset = val_dataset.cache().batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.cache().batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)


base_model = NASNetLarge(weights='imagenet', include_top=False, input_shape=(240, 320, 3))  


for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.6)(x)  
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)  
x = Dropout(0.6)(x)  
x = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(x)
predictions = Dense(4, activation='linear')(x)


model = Model(inputs=base_model.input, outputs=predictions)


optimizer = Adam(learning_rate=1e-4)  
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])


early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)  
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-7)  
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')


history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=10,  
                    callbacks=[early_stopping, reduce_lr, checkpoint])


model.load_weights('best_model.keras')




def plot_history(history, model_name):
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['loss'], label='train_loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.title(f'{model_name} Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    plt.figure(figsize=(12, 6))
    plt.plot(history.history['mae'], label='train_mae')
    plt.plot(history.history['val_mae'], label='val_mae')
    plt.title(f'{model_name} MAE')
    plt.xlabel('Epochs')
    plt.ylabel('MAE')
    plt.legend()
    plt.show()

plot_history(history, 'NASNetLarge Fine-tuned Model')


Epoch 1/10
 9/71 ━━━━━━━━━━━━━━━━━━━━ 18:01 17s/step - loss: 6.1715 - mae: 1.4621

KeyboardInterrupt: 